# Homework #1 - Drive duration prediction



In [ ]:
!python -V

In [ ]:
import pickle

import matplotlib.pyplot as plt
import pandas as pd
import seaborn as sns
from sklearn.feature_extraction import DictVectorizer
from sklearn.linear_model import (
    LinearRegression,
    Lasso,
    Ridge,
)
from sklearn.metrics import root_mean_squared_error

In [ ]:
df = pd.read_parquet('../../data/yellow_tripdata_2023-01.parquet')
print(df.info())
df.head()

Used columns

In [ ]:
COL_DURATION = "duration"
COL_PICKUP_LOC = 'PULocationID'
COL_DROPOFF_LOC = 'DOLocationID'

In [ ]:
df[COL_DURATION] = df.tpep_dropoff_datetime - df.tpep_pickup_datetime
df[COL_DURATION] = df[COL_DURATION].apply(lambda td: td.total_seconds() / 60)

print(f"base metrics for duration (min):")
print(f"* mean:{df[COL_DURATION].mean()}")
print(f"* std:{df[COL_DURATION].std() }")

df.describe()

In [ ]:
df_ = df[(df[COL_DURATION] >= 1) & (df[COL_DURATION] <= 60)]

print(f"raw data rows: {len(df)}")
print(f"cleaned data rows: {len(df_)}")
print(f"rate cleaned rows: {100 * len(df_) / len(df)}%")

In [ ]:
categorical = [COL_DROPOFF_LOC, COL_PICKUP_LOC]
df[categorical] = df[categorical].astype(str)

df.head()

In [ ]:
train_dicts = df[categorical].to_dict(orient='records')

dv = DictVectorizer()
X_train = dv.fit_transform(train_dicts)

print(X_train.shape)

In [ ]:
target = COL_DURATION
y_train = df[target].values

print(len(y_train))

In [ ]:
lr = LinearRegression()
lr.fit(X_train, y_train)

y_pred = lr.predict(X_train)

root_mean_squared_error(y_train, y_pred)

In [ ]:
sns.histplot(y_pred, label='prediction')
sns.histplot(y_train, label='actual')

plt.legend()

In [ ]:
def read_dataframe(filename):
    if filename.endswith('.csv'):
        df = pd.read_csv(filename)

        df.tpep_dropoff_datetime = pd.to_datetime(df.tpep_dropoff_datetime)
        df.tpep_pickup_datetime = pd.to_datetime(df.tpep_pickup_datetime)
    elif filename.endswith('.parquet'):
        df = pd.read_parquet(filename)

    df[COL_DURATION] = df.tpep_dropoff_datetime - df.tpep_pickup_datetime
    df[COL_DURATION] = df[COL_DURATION].apply(lambda td: td.total_seconds() / 60)

    df = df[(df[COL_DURATION] >= 1) & (df[COL_DURATION] <= 60)]

    categorical = [COL_DROPOFF_LOC, COL_PICKUP_LOC]
    df[categorical] = df[categorical].astype(str)
    
    return df

In [ ]:
df_val = read_dataframe('../../data/yellow_tripdata_2023-02.parquet')

In [ ]:
len(df_val)

In [ ]:
val_dicts = df_val[categorical].to_dict(orient='records')
X_val = dv.transform(val_dicts)

In [ ]:
y_val = df_val[target].values

In [ ]:
y_pred = lr.predict(X_val)

root_mean_squared_error(y_val, y_pred)